In [40]:
import json
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString


def xml_to_json(xml_string):
    def parse_element(element):
        node = {"type": element.tag}
        if element.attrib:
            node.update(element.attrib)
        children = list(element)
        if children:
            node["nodes"] = [parse_element(child) for child in children]
        elif element.text and element.text.strip():
            node["parameters"] = {"text": element.text.strip()}
        return node

    root = ET.fromstring(xml_string)
    main_tree_to_execute = root.attrib.get("main_tree_to_execute", "MainTree")
    main_tree = None

    # Find the BehaviorTree node that matches the main_tree_to_execute
    for child in root:
        if child.attrib.get("ID") == main_tree_to_execute:
            main_tree = parse_element(child)
            break

    # Return only the core nodes (remove BehaviorTree wrapper)
    return main_tree["nodes"][0]  # The core behavior tree


def json_to_xml(json_obj):
    def build_element(node):
        element = ET.Element(node["type"])
        for key, value in node.items():
            if key == "nodes":
                for child in value:
                    element.append(build_element(child))
            elif key != "type":
                element.set(key, str(value))
        return element

    # Wrap the core JSON in a BehaviorTree and root
    behavior_tree = {
        "type": "BehaviorTree",
        "ID": "MainTree",
        "nodes": [json_obj],  # Wrap the core tree as nodes
    }
    root_element = ET.Element("root", {"main_tree_to_execute": "MainTree"})
    behavior_tree_element = build_element(behavior_tree)
    root_element.append(behavior_tree_element)

    # Pretty-print the XML
    rough_string = ET.tostring(root_element, "utf-8")
    return parseString(rough_string).toprettyxml(indent="    ")


In [41]:
xml_string = '''
<root main_tree_to_execute="MainTree">
  <BehaviorTree ID="MainTree">
    <Sequence name="RetrieveFoodSequence">
      <!-- Locate the food -->
      <Retry retries="3">
        <Locate object="apple" location="kitchen" method="camera_scan" />
      </Retry>
      
      <!-- Fallback to ask for help if food cannot be located -->
      <Fallback name="LocateFoodFallback">
        <Locate object="apple" location="kitchen" method="camera_scan" />
        <AskForHelp message="Unable to locate the apple. Please assist." />
      </Fallback>
      
      <!-- Navigate to the food's location -->
      <Retry retries="2">
        <NavigateTo location="kitchen_table" mode="safe" speed="medium" />
      </Retry>

      <!-- Pick up the food -->
      <Retry retries="2">
        <Pick object="apple" grip_strength="medium" precision="high" />
      </Retry>
      
      <!-- Navigate to the drop-off location -->
      <Retry retries="2">
        <NavigateTo location="dining_table" mode="direct" speed="fast" />
      </Retry>

      <!-- Place the food on the table -->
      <Fallback name="PlaceFoodFallback">
        <Place object="apple" surface="dining_table" orientation="upright" alignment="center" />
        <AskForHelp message="Unable to place the apple. Please assist." />
      </Fallback>
    </Sequence>
  </BehaviorTree>
</root>

'''

# Convert XML to JSON
json_obj = xml_to_json(xml_string)
# print(json_obj)

# Convert JSON back to XML
xml_output = json_to_xml(json_obj)
xml_output = xml_output.replace('\n', '').replace('<?xml version="1.0" ?>', '')

# compare the two xml strings
xml_string = xml_string.replace('\n', '')
print(xml_string)
print(xml_output)
print(xml_string == xml_output)

print(str(json_obj))

<root main_tree_to_execute="MainTree">  <BehaviorTree ID="MainTree">    <Sequence name="RetrieveFoodSequence">      <!-- Locate the food -->      <Retry retries="3">        <Locate object="apple" location="kitchen" method="camera_scan" />      </Retry>            <!-- Fallback to ask for help if food cannot be located -->      <Fallback name="LocateFoodFallback">        <Locate object="apple" location="kitchen" method="camera_scan" />        <AskForHelp message="Unable to locate the apple. Please assist." />      </Fallback>            <!-- Navigate to the food's location -->      <Retry retries="2">        <NavigateTo location="kitchen_table" mode="safe" speed="medium" />      </Retry>      <!-- Pick up the food -->      <Retry retries="2">        <Pick object="apple" grip_strength="medium" precision="high" />      </Retry>            <!-- Navigate to the drop-off location -->      <Retry retries="2">        <NavigateTo location="dining_table" mode="direct" speed="fast" />      </Retr

In [42]:
json_string = """{
  "type": "Sequence",
  "name": "RetrieveAndReturnWaterBottleSequence",
  "nodes": [
    {
      "type": "Locate",
      "object": "water_bottle",
      "location": "refrigerator",
      "method": "camera_scan"
    },
    {
      "type": "NavigateTo",
      "location": "kitchen_table",
      "mode": "direct",
      "speed": "fast"
    },
    {
      "type": "Pick",
      "object": "water_bottle",
      "grip_strength": "medium",
      "precision": "high"
    },
    {
      "type": "NavigateTo",
      "location": "kitchen_table",
      "mode": "reverse",
      "speed": "fast"
    },
    {
      "type": "PutDown",
      "object": "water_bottle",
      "surface": "kitchen_table",
      "orientation": "upright"
    }
  ]
}"""

json_obj = json.loads(json_string)
print(json_obj)

xml_output = json_to_xml(json_obj)
xml_output #= xml_output.replace('\n', '').replace('<?xml version="1.0" ?>', '')
print(xml_output)



{'type': 'Sequence', 'name': 'RetrieveAndReturnWaterBottleSequence', 'nodes': [{'type': 'Locate', 'object': 'water_bottle', 'location': 'refrigerator', 'method': 'camera_scan'}, {'type': 'NavigateTo', 'location': 'kitchen_table', 'mode': 'direct', 'speed': 'fast'}, {'type': 'Pick', 'object': 'water_bottle', 'grip_strength': 'medium', 'precision': 'high'}, {'type': 'NavigateTo', 'location': 'kitchen_table', 'mode': 'reverse', 'speed': 'fast'}, {'type': 'PutDown', 'object': 'water_bottle', 'surface': 'kitchen_table', 'orientation': 'upright'}]}
<?xml version="1.0" ?>
<root main_tree_to_execute="MainTree">
    <BehaviorTree ID="MainTree">
        <Sequence name="RetrieveAndReturnWaterBottleSequence">
            <Locate object="water_bottle" location="refrigerator" method="camera_scan"/>
            <NavigateTo location="kitchen_table" mode="direct" speed="fast"/>
            <Pick object="water_bottle" grip_strength="medium" precision="high"/>
            <NavigateTo location="kitchen_ta

In [43]:
json_text2['type']

'Sequence'

In [46]:
json_text2 = {'type': 'Sequence', 'name': 'RetrieveBeerSequence', 'nodes': [{'type': 'NavigateTo', 'location': 'beer-fridge', 'mode': 'safe', 'speed': 'medium'}, {'type': 'Retry', 'retries': '3', 'nodes': [{'type': 'OpenBeerFridgeDoor', 'method': 'handle Pull'}]}, {'type': 'Fallback', 'name': 'OpenBeerFridgeFallback', 'nodes': [{'type': 'OpenBeerFridgeDoor', 'method': 'handle Pull'}, {'type': 'AskForHelp', 'message': 'Unable to open the beer fridge door. Please assist.'}]}, {'type': 'Retry', 'retries': '2', 'nodes': [{'type': 'RetrieveBeer', 'object': 'beer-can'}]}, {'type': 'Fallback', 'name': 'PlaceBeerFallback', 'nodes': [{'type': 'RetrieveBeer', 'object': 'beer-can'}, {'type': 'AskForHelp', 'message': 'Unable to retrieve the beer. Please assist.'}]}]}
# json_obj2 = json.loads(json_text2)
#print(json_obj2)

from goat_planner.goat_controller import json_to_xml

xml_output2 = json_to_xml(json_text2)
print(xml_output2)

TypeError: string indices must be integers

In [47]:
xml_test = """<?xml version="1.0" ?>
<root main_tree_to_execute="MainTree">
    <BehaviorTree ID="MainTree">
        <Sequence name="MainSequence">
            <Wait duration="2.0"/>
            <Navigate x="0.0" y="6.0"/>
            <Wait duration="2.0"/>
        </Sequence>
    </BehaviorTree>
</root>
"""

json_obj = xml_to_json(xml_test)
print(json_obj)

{'type': 'Sequence', 'name': 'MainSequence', 'nodes': [{'type': 'Wait', 'duration': '2.0'}, {'type': 'Navigate', 'x': '0.0', 'y': '6.0'}, {'type': 'Wait', 'duration': '2.0'}]}


In [48]:
from goat_planner.goat_state import GoatState

goat_state = GoatState()
new_tree = {'type': 'Sequence', 'nodes': [{'type': 'Wait', 'duration': '1.0'}]}
goat_state.update_behavior_tree(new_tree)